In [2]:
doc0 = ['abc', 'bcbcd', 'agebc', 'adebc']
doc1 = ['fegd', 'fcgecd', 'fgged']
doc2 = ['bacd', 'acabd', 'badce', 'fg']
doc3 = ['gdefa', 'gbaccd', 'adefeg', 'gafedbc', 'bcbcd']
docs = [doc0, doc1, doc2, doc3]
docPairs = [0,0,0,0]

In [4]:
letterCounts = {}
idx = 0
for d in docs:
    for s in d:
        for i in s:
            if i in letterCounts:
                letterCounts[i] = (letterCounts[i][0], letterCounts[i][1] + 1)
            else:
                letterCounts[i] = (idx, 1)
                idx += 1
letterCounts

{'a': (0, 11),
 'b': (1, 12),
 'c': (2, 15),
 'd': (3, 13),
 'g': (4, 10),
 'e': (5, 10),
 'f': (6, 7)}

In [9]:
totalLetters = sum(v[1] for v in letterCounts.values())
totalLetters

78

In [10]:
tensor = {}
for docNum,doc in enumerate(docs):
    for s in doc:
        for i in range(len(s) - 1):
            for j in range(i + 1, len(s)):
                key = (s[i], s[j], docNum)
                docPairs[docNum] += 1
                if key in tensor:
                    tensor[key] += 1
                else:
                    tensor[key] = 1

tensor now is an object that for every index (i,j,k) 
has the ABSOLUTE number of occurances of word J AFTER word I in document K
These word pairs are only formed inside sentences. So if word I is the last word in sentence 1,
and word J is the first word in sentence 2, then the absolute count isn't increased.
however, if word I is the first word in sentence 1 and word J is the last word in sentence 1, 
then they are still counted

Other methods include:

1) only counting directly adjacent words

2) giving a pair a fraction of a score based on word distance

3) 2, but limit to only 5 (or 10 or 3 whatever it doesn't really matter) words

4) 1, but with the constraints of 3

5) More ways?

In [11]:
docPairs

[33, 31, 27, 71]

In [12]:
numPairs = sum(list(tensor.values()))
numPairs #total number of pairs of words, on the order of n^2 sentences

162

Moving on to Normalization:
This is one of the ways I'm going to do it below, but there are many others that could possibly work better

The way I'm going to do it now is by PMI: ln( p(x,y,z) / (p(x)p(y)p(z)) )

p(x,y,z) is the probability of finding the pair x,y in document z
    this is calculated by taking the number of pairs in document z, and then finding the proportion of the pair (x,y) in the total number of pairs in doc z
    
    p(x) is #occurances of word x/ # of occurances of all words
    
    p(y) is the same
    
    p(z) is just 1/#documents (i think)
    
properties of this normalization (before the ln):
sum(p(x,y,z)) for a fixed z is 1

Another PMI that could work:
p(x,y,z)/p(x,y)p(x)p(y)

p(x,y,z) is the same

p(x,y) is the probability of finding pair (x,y) in all documents combined
p(x),p(y) are the same

In [13]:
from numpy import log as ln

In [18]:
sumNums = [0,0,0,0];
normTensor = {};
for k in tensor.keys():
    v = tensor[k] #number of times the pair w1,w2 showed up in the document
    w1 = k[0]
    w2 = k[1]
    d = k[2];
    #p(x,y)
    numerator = v/docPairs[d]
    #p(x)p(y)
    pw1 = letterCounts[w1][1]/totalLetters
    pw2 = letterCounts[w2][1]/totalLetters
    denominator = pw1 * pw2
    newKey = (letterCounts[k[0]][0],letterCounts[k[1]][0], k[2])
    normTensor[newKey] = ln(numerator/denominator)
    sumNums[d] += numerator
normTensor

{(0, 1, 0): 1.4327204579944421,
 (0, 2, 0): 1.2095769066802324,
 (1, 2, 0): 1.815712710250548,
 (1, 1, 0): 0.2470967923367026,
 (1, 3, 0): 0.8602012652231116,
 (2, 1, 0): 0.0239532410224928,
 (2, 2, 0): -0.19919031029171697,
 (2, 3, 0): 0.6370577139089019,
 (0, 4, 0): 0.516429726120287,
 (0, 5, 0): 1.2095769066802324,
 (4, 5, 0): 0.6117399059246121,
 (4, 1, 0): 0.42941834913065724,
 (4, 2, 0): 0.2062747978164476,
 (5, 1, 0): 1.1225655296906025,
 (5, 2, 0): 0.8994219783763929,
 (0, 3, 0): 0.2540654616527961,
 (3, 5, 0): 0.349375641457121,
 (3, 1, 0): 0.16705408466316624,
 (3, 2, 0): -0.05608946665104347,
 (6, 5, 1): 2.129547495512788,
 (6, 4, 1): 2.417229567964569,
 (6, 3, 1): 1.867183231045297,
 (5, 4, 1): 0.6742602629059461,
 (5, 3, 1): 1.5105082871065645,
 (4, 3, 1): 1.7981903595583455,
 (6, 2, 1): 1.318617279296459,
 (2, 4, 1): 0.2687951547977816,
 (2, 5, 1): 0.2687951547977816,
 (2, 2, 1): -0.13666995331038298,
 (2, 3, 1): 0.6995780708902358,
 (4, 5, 1): 1.7728725515740558,
 (4, 2,

In [19]:
sumNums #should be all 1's

[0.9999999999999996,
 0.9999999999999999,
 0.9999999999999996,
 0.9999999999999996]